In [1]:
import torch    
import torch.nn as nn
import pytorch_lightning as pl

/home/user/miniconda3/envs/open-ce/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


A notebook to test whether it's possible to use a pytorch linear layer with the same parameter as an embedding layer. 

We're hoping ot see that we can instantiate both. Then as the embedding model is trained, the linear layer also moves. and same vise versa. 

This is a test to see if we can use the same parameter for both an embedding layer and a linear layer.

In [12]:
import numpy as np
import operator
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from functools import reduce
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import numpy as np
class PTLModule(pl.LightningModule):
    def __init__(self,
                batch_size=16,
                learning_rate=0.001,
                logitsversion=17,
                n=6,
                normlogits=False,
                logvariance=False,
                maskLosses=0,):
        super().__init__(
           
        )
        self.save_hyperparameters()
        self.emb=nn.Embedding(1000, 32) #1000 items in the vocab, 32 dimensional embedding
        self.linearEmbedding=nn.Linear(1000 , 32 ) # 1000 items in the vocab, 32 dimensional embedding
        self.linearEmbedding.weight=nn.Parameter(self.emb.weight.T) 
        self.layer1 = nn.Linear(32, 128)
        self.layer2 = nn.Linear(128, 64)
        from model.nargsLossCalculation import get_loss_fn 
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))
        self.batch_size=batch_size
        self.learning_rate=learning_rate
        self.calculate_loss=get_loss_fn(logitsversion=logitsversion,norm=normlogits,log=logvariance)
        self.n=n
        self.maskLoss=maskLosses
        self.maskloss=torch.nn.MSELoss(reduction='none')

        torch.autograd.set_detect_anomaly(True)
        #with torch.no_grad():
       
    def forward(self, x):
        
        x = self.layer1(x)
        x = self.layer2(x)
        return x
    def setup(self, stage):
        self.train_dataset = torch.utils.data.TensorDataset(torch.randint(0, 1000, (10000,)))
        self.val_dataset = torch.utils.data.TensorDataset(torch.randint(0, 1000, (1000,)))
        self.test_dataset = torch.utils.data.TensorDataset(torch.randint(0, 1000, (1000,)))
        # self.labels=torch.diag_embed(torch.diag_embed(torch.diag_embed(torch.diag_embed(torch.diag_embed(torch.ones(self.batch_size,dtype=torch.float,device=self.device))))))
        # self.labels=torch.nan_to_num(self.labels)

        #B,N=self.batch_size,6
        #Views=torch.diag_embed(torch.ones(N,dtype=torch.long)*B-1)+1
        #Lossmask=torch.sum(reduce(torch.add,list(map(lambda Arr: torch.nn.functional.one_hot(torch.arange(B).view(*Arr),num_classes=B),Views.tolist()))).pow(4),dim=-1)

        #self.masks=torch.unique(torch.flatten(Lossmask,0,-1),dim=0,sorted=False)

        self.alpha=None
        from model.LossCalculation import get_loss_sum
        self.meanloss=get_loss_sum(0)
       
        from model.LossCalculation import get_loss_calc

        self.loss=get_loss_calc(reduction='sum',ver=self.maskLoss,mask=torch.ones([1],device=self.device))
    def train_dataloader(self,batch_size=32):
      
        import torch.utils.data.dataloader as dataloader


        return dataloader.DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True,num_workers=8,drop_last=True)
    def val_dataloader(self,batch_size=32):
      
        import torch.utils.data.dataloader as dataloader


        return dataloader.DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=True,num_workers=8,drop_last=True)
    def test_dataloader(self,batch_size=32):
        
        import torch.utils.data.dataloader as dataloader
    
    
        return dataloader.DataLoader(self.test_dataset,batch_size=self.batch_size,shuffle=True,num_workers=8,drop_last=True)
    
    def training_step(self, batch, batch_idx):
        #batch shape is n
        #print("batch",batch)
        n=6
        x=self.emb(batch[0]) # should be Bxf 
        nx=[self(x+torch.randn_like(x))]*n # should be Bxf
        # labels=self.label[:(im.shape[0]),:(im.shape[0]),:(im.shape[0]),:(im.shape[0]),:(im.shape[0]),:(im.shape[0])].to(self.device,non_blocking=True) 

        logits=self.calculate_loss(*nx).mul(torch.exp(self.logit_scale))
        #print("logits",logits.shape)
        #self.log("first logit",logits[0,0,0,0,0,0],enable_graph=False)
        #self.log("BAD logit",logits[1,2,3,4,5,0],enable_graph=False)
        # The idea is that good logits are 1s,   bad should be -1s... so if logits are coming back as ~6000....
        #  Option 1: divide down.
        #  Option 2: 1- output...
        # option 3: logarithmic functions? 
        #print("logits",logits.shape)
        #print("labels",labels.shape)
        labels=torch.diag_embed(torch.diag_embed(torch.diag_embed(torch.diag_embed(torch.diag_embed(torch.ones_like(batch[0],dtype=torch.float))))))
        labels=torch.nan_to_num(labels)
        #print("labels",labels.shape)
        lossim = self.loss(logits, labels,alpha=self.alpha)
            
        
        loss1 = self.loss(self.calculate_loss(*nx).mul(torch.exp(self.logit_scale)).permute(1,2,3,4,5,0), labels,alpha=self.alpha)
        loss2 = self.loss(self.calculate_loss(*nx).mul(torch.exp(self.logit_scale)).permute(2,3,4,5,0,1), labels,alpha=self.alpha)
        loss3 = self.loss(self.calculate_loss(*nx).mul(torch.exp(self.logit_scale)).permute(3,4,5,0,1,2), labels,alpha=self.alpha)
        loss4 = self.loss(self.calculate_loss(*nx).mul(torch.exp(self.logit_scale)).permute(4,5,0,1,2,3), labels,alpha=self.alpha)
        loss5 = self.loss(self.calculate_loss(*nx).mul(torch.exp(self.logit_scale)).permute(5,0,1,2,3,4), labels,alpha=self.alpha)
        loss=self.meanloss(I=[lossim],T=[loss1,loss2,loss3,loss4,loss5]).mean()
        self.log("loss",loss,enable_graph=False)
        return  {"loss":loss, "labels":batch[0], "embs":nx[0]}  

    def training_epoch_end(self, outputs):
        #concatenate outputs,
        # 
        # create linear regression model
        # 
        # fit model
        # 
        # get score
        alllabels=torch.cat([x['labels'] for x in outputs],dim=0)
        allembs=torch.cat([x['embs'] for x in outputs],dim=0)
        # print("all labels",alllabels.shape)
        # print("all embs",allembs.shape)
        from sklearn.linear_model import LogisticRegression

        reg =LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=0, n_jobs=-1)
        reg.fit(allembs.detach().cpu().numpy(), alllabels.detach().cpu().numpy())
        
        self.log("score",reg.score(allembs.detach().cpu().numpy(), alllabels.detach().cpu().numpy()),on_step=False,on_epoch=True,prog_bar=True,logger=True)
        #store score to be used in numpy plot

    def validation_step(self,batch,batch_idx):
        #heck that converting batch to one hot is with 1000 classes is working
        one_hot=torch.nn.functional.one_hot(batch[0],num_classes=1000).float()
        #Bx1000 
        linear=one_hot @self.emb.weight#Bx 32  
        success=torch.allclose(linear, self.emb(batch[0]))
        #return 1 or 0 for success
        
        self.log("test success",success,on_step=False,on_epoch=True,prog_bar=True,logger=True)
        return {"success":1 if success else 0}
    def validation_epoch_end(self,outputs):
        success=reduce(operator.add,[x['success'] for x in outputs])/len(outputs)
        self.log("test success",success,on_step=False,on_epoch=True,prog_bar=True,logger=True)
    
    def configure_optimizers(self):
        
        optimizer = torch.optim.AdamW(
            [p for p in self.parameters()], lr=self.hparams.learning_rate, eps=10e-8,
            #weight_decay=0.1,
            #betas=(0.9, 0.95),
            )
        return [optimizer]

model=PTLModule()
trainer = Trainer(
gpus=1,
max_epochs=20,
logger=TensorBoardLogger("tb_logs", name="EmbLinearTest{}".format(0),version=f"{0}"),
auto_scale_batch_size="binsearch",
auto_lr_find=True)

trainer.tune(model)

trainer.fit(model)
#trainer.test(model)



/home/user/miniconda3/envs/open-ce/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:474: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/user/miniconda3/envs/open-ce/lib/python3.9/site-packages/pytorch_lightning/callbacks/batch_size_finder.py:161: UserWarning: Field `model.batch_size` and `mod

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x1000 and 32x1000)